In [1]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Click the "Load More" button for scraping more news

In [2]:
def extendPage():
    while True:
        try:
            try:
                # Just to locate to the Load More button to be clicked
                load_more = driver.find_element_by_class_name('quicklinks')
                actions = ActionChains(driver)
                actions.move_to_element(load_more).perform()
            except NoSuchElementException:
                print('Skip')
                break

            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "loadMorestories"))).click()
            break
        except ElementClickInterceptedException as a:
            print("Try again")
            pass
        except TimeoutException as b:
            print("Try again")
            pass
        except NoSuchElementException as c:
            print("Try again")
            pass

### Iterate the given list to text or url

In [3]:
def toList(given_list, condition):
    convert = []
    for i in given_list:
        if condition == 'link':
            convert.append(i.get_attribute('href'))
        else:
            convert.append(i.text)
    non_empty_ele = [j for j in convert if j]
    return non_empty_ele

### Activate chrome driver

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.thestar.com.my/')

### Extract all types of news

In [5]:
types = driver.find_elements_by_xpath(".//li[@class = 'dropdown']")
all_type = toList(types, 'text')
all_type.remove('Videos')
all_type.remove('Photos')
print(all_type)

['StarPlus', 'News', 'Asean+', 'Business', 'Sport', 'Metro', 'Lifestyle', 'Food', 'Tech', 'Education', 'Opinion', 'StarPicks']


In [6]:
links_list = []
news_list = []

### Browse through every type of webpage, scrape the links and store into a list


In [7]:
for element in all_type:
    xpath = f"//a[contains(text(), '{element}')]"
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))).click()
    print(xpath)
    extendPage()

    travel1 = driver.find_elements_by_xpath(".//main//div[5]//h2/a")
    if element.lower() == 'starplus' or element.lower() == 'business' or element.lower() == 'lifestyle' or element.lower() == 'opinion':
        travel2 = driver.find_elements_by_xpath(".//main//div[6]//h2/a")
    else:
        travel2 = driver.find_elements_by_xpath(".//main//div[6]/div/div/div[2]//h2/a")

    links_list1 = toList(travel1, 'link')
    links_list2 = toList(travel2, 'link')
    links_list = links_list + links_list1 + links_list2
    print(len(links_list))

driver.close()

//a[contains(text(), 'StarPlus')]
Skip
17
//a[contains(text(), 'News')]
57
//a[contains(text(), 'Asean+')]
Skip
87
//a[contains(text(), 'Business')]
127
//a[contains(text(), 'Sport')]
167
//a[contains(text(), 'Metro')]
207
//a[contains(text(), 'Lifestyle')]
Skip
243
//a[contains(text(), 'Food')]
273
//a[contains(text(), 'Tech')]
Skip
303
//a[contains(text(), 'Education')]
Skip
317
//a[contains(text(), 'Opinion')]
Skip
326
//a[contains(text(), 'StarPicks')]
Skip
343


### Remove duplicates from list


In [8]:
filtered_links = []
[filtered_links.append(x) for x in links_list if x not in filtered_links]
print(f'No. of non-duplicated link: {len(filtered_links)}')

No. of non-duplicated link: 301


### Considering the Food category contains some recipes which will lead to another web page, therefore I decided to remove non The Star link here


In [9]:
for i in range(5):
    for i, ele in enumerate(filtered_links):
        if not ele.startswith('https://www.thestar.com.my/'):
            del filtered_links[i]
print(f'No. of non-TheStar link: {len(filtered_links)}')
filtered_links

No. of non-TheStar link: 295


['https://www.thestar.com.my/news/nation/2021/06/09/interactive-heres-what-vaccinating-malaysias-children-against-covid-19-could-involve',
 'https://www.thestar.com.my/news/nation/2020/01/26/the-rapid-global-spread-of-the-novel-coronavirus---total-cases-so-far',
 'https://www.thestar.com.my/news/nation/2021/04/24/interactive-how-malaysias-fight-against-covid-19-is-progressing-updated-weekly',
 'https://www.thestar.com.my/news/nation/2021/06/02/interactive-more-covid-19-patients-are-being-brought-in-dead---why-this-happens-and-what-needs-to-be-done',
 'https://www.thestar.com.my/news/nation/2021/05/25/interactive-malaysian-twitter-users-rank-near-the-top-globally-in-support-of-palestine',
 'https://www.thestar.com.my/news/nation/2020/03/23/covid-19-current-situation-in-malaysia-updated-daily',
 'https://www.thestar.com.my/news/nation/2021/05/23/interactive-suffering-in-silence',
 'https://www.thestar.com.my/news/nation/2021/05/16/interactive-teachers-a-class-above-in-difficult-times',
 

### Browse through each url and crawl the news content

In [10]:
for link in filtered_links:
    driver = webdriver.Chrome()
    url = f"{link}"
    print(link)
    driver.get(url)

    try:
        title = driver.find_element_by_tag_name('h1').text
    except NoSuchElementException:
        driver.close()
        continue

    try:
        author = driver.find_elements_by_xpath(".//div[@class = 'credit__authors']/a")
        author = toList(author, 'text')
        author = re.sub(",", " ", '，'.join(author))
    except NoSuchElementException:
        author = ''
        pass

    try:
        date = driver.find_element_by_xpath(".//p[@class = 'date']").text
    except NoSuchElementException:
        date = ''
        pass

    try:
        timestamp = driver.find_element_by_xpath(".//time[@class = 'timestamp']").text
    except NoSuchElementException:
        timestamp = ''
        pass

    # Extract all the text of the news content
    parent = driver.find_element_by_id('story-body')
    content = list(parent.text.split('\n'))

    # The following 4 statements are to extract all irrelevant content such as chart and image caption
    chart_text = parent.find_elements_by_xpath(".//div[contains(@class, 'flourish')]")
    inline_text = parent.find_elements_by_xpath(".//span[contains(@class, 'inline-caption')]")
    caption_text = parent.find_elements_by_xpath(".//p[contains(@class, 'caption')]")
    extra_text = chart_text + inline_text + caption_text

    full_content = toList(extra_text, 'text')
    full_content = list(set(full_content))

    # Filter out irrelevant content
    for index, i in enumerate(content):
        for j in full_content:
            if i == j:
                content[index] = ''

    # Remove the enter and empty field from the extracted news content
    real_content = ''
    for element in content:
        real_content += element

    # Determine the type of the news by using the url
    for category in all_type:
        str = f"https://www.thestar.com.my/{category.lower()}"
        result = re.search(str, link)
        if result is not None:
            break

    print(date)
    print(timestamp)
    print(title)
    print(category)
    print(author)
    print(real_content)

    news_items = {'Date': date, 'Time': timestamp, 'Type': category, 'Author': author, 'Title': title, 'Content': real_content}
    news_list.append(news_items)
    print(len(news_list))
    driver.close()
    time.sleep(2)

https://www.thestar.com.my/news/nation/2021/06/09/interactive-heres-what-vaccinating-malaysias-children-against-covid-19-could-involve
Wednesday, 09 Jun 2021
7:00 AM MYT
INTERACTIVE: Here’s what vaccinating Malaysian children against Covid-19 could involve
News
RAZAK AHMAD，DIYANA PFORDTEN
PETALING JAYA: Malaysia is expected to announce its decision soon on whether children should be included in the country’s Covid-19 immunisation programme.The current plan currently only covers those aged 18 and above, but a growing number of other countries have already started to vaccinate children.The government is currently studying whether Malaysia should follow suit. If the country were to decide to vaccinate its children, what would it involve?According to the National Covid-19 Immunisation Programme document, Malaysia currently aims to vaccinate at least 23.6 million people aged 18 and above.Calculations based on the most recent available figures show that vaccinating children aged 12 and above

https://www.thestar.com.my/news/nation/2021/05/25/interactive-malaysian-twitter-users-rank-near-the-top-globally-in-support-of-palestine
Tuesday, 25 May 2021
7:00 AM MYT
INTERACTIVE: Malaysian Twitter users rank near the top globally in support of Palestine
News
DIYANA PFORDTEN，RAZAK AHMAD
PETALING JAYA: Computer science graduate Muadz Halim normally tweets about football and Japanese manga, but when Israel attacked Palestine earlier this month, his focus expanded.The 23-year-old from Perak began to retweet and use pro-Palestine hashtags from early May, at a rate of 80 to 120 times each day."I hope that by doing this, I can at least raise awareness among my social media followers that the Palestinians are suffering and that we need to always support and show them solidarity."My greatest hope is that Palestine will be free and that there will be no more suffering,” said Muadz, who has racked up a total of 2,707 mentions and retweets on the topic.He is one of an army of local social medi

https://www.thestar.com.my/news/nation/2021/05/16/interactive-teachers-a-class-above-in-difficult-times
Sunday, 16 May 2021
7:00 AM MYT
INTERACTIVE: Teachers a class above in difficult times
News
REBECCA RAJAENDRAM
TOUGH times call for strong characters, especially if you are a teacher trying to educate students living in hard-to-reach areas during a pandemic.Though schools had to close when the movement control order (MCO) began, education was not brought to a halt.It is easy for a teacher to teach when the conditions are ideal - the classroom is comfortable, there are enough tables and chairs, every student has a book, stationery and can they see the board clearly.But in rural schools, more often than not, the situation is far from ideal. Little, if at all, is readily available.The National Union of the Teaching Profession (NUTP) secretary-general Harry Tan said it was passion and responsibility that drove these teachers to do everything within their power to carry out their duty.Wit

https://www.thestar.com.my/news/nation/2021/05/06/interactive-vaccine-passports---the-what-how-why-and-why-not
Thursday, 06 May 2021
7:00 AM MYT
INTERACTIVE: Vaccine passports – the what, how, why and why not
News
DIYANA PFORDTEN
PETALING JAYA: Countries around the world are exploring ways to show proof of an individual's Covid-19 immunity in order to allow travel and other easing of social restrictions.Several countries have initiated travel passports, which are Covid-19 vaccine certificates or any physical or digital document that would prove you've received both doses (or one dose for certain vaccines) of any of the available Covid-19 vaccines.In some countries, vaccine certificates can also be issued to those who have recently tested negative for Covid-19 or have built immunity against it after having been infected.All this would allow citizens to travel among participating countries or be granted exemptions to certain Covid-19 restrictions in their own country.A recent survey by m

https://www.thestar.com.my/news/nation/2021/04/24/interactive-myanmar-at-a-crossroads
Saturday, 24 Apr 2021

INTERACTIVE: Myanmar at a Crossroads
News

Myanmar at a Crossroadsby MAZWIN NIK ANIS and JOSEPH KAOS, JRAP PhotoWHEN the 10 members of Asean meet for a special summit in Jakarta today, the most urgent and immediate task at hand is to resolve the political crisis in Myanmar, which has so far seen over 700 people killed in protests across the country.The fact that the summit, the first physical meeting amidst a pandemic, is taking place speaks volume of the urgent matter at hand and will see the attendance of heads of government, except for Thai Prime Minister Prayuth Chan o-cha who has confirmed that he will be skipping the meet.Thailand will instead be represented by Deputy Prime Minister and Foreign Minister Don Pramudwinai.If the head of Myanmar’s military government, Senior General Min Aung Hlaing, does indeed show up for the meet, this will be his first known foreign trip si

...

https://www.thestar.com.my/starpicks/2021/06/05/revolutionise-change
Saturday, 05 Jun 2021

Revolutionise change
StarPicks

Revolutionise changeExperience the new BMW 5 Series* Disclaimer: Model shown may vary based on local specificationsTOPBuilt with Shorthand
285
https://www.thestar.com.my/starpicks/2021/06/04/5g-ready-redmi-note-10-5g-is-here-at-only-rm799
Friday, 04 Jun 2021
2:18 PM MYT
5G-ready Redmi Note 10 5G is here, at only RM799
StarPicks

THE highly anticipated Redmi Note 10 5G has reached Malaysia’s shores, being Xiaomi’s latest best budget smartphone with 5G connectivity while delivering flagship-standard performance.Powered by the MediaTek Dimensity 700, this Xiaomi mid-range device provides flagship-level power-efficiency with its 7nm manufacturing node, dual 5G SIM supported system and integrated modem.Silky smooth experienceFeaturing a captivating and immersive 90Hz 6.5 inch AdaptiveSync DotDisplay, the Redmi Note 10 5G is capable of adjusting to content automatically

https://www.thestar.com.my/starpicks/2021/06/04/world-fiabci-award-for-the-12-waves
Friday, 04 Jun 2021
8:23 AM MYT
World FIABCI Award for The 12 Waves
StarPicks

THE 12 Waves followed up its win in the Industrial Category at the FIABCI Malaysia Property Awards (MPA) in November 2019, by bagging the Silver Award in the Industrial Category of the 2021 FIABCI World Prix d’Excellence Awards that were announced online on June 2.The only Malaysian development that made it into the winners’ list for the world finals, The 12 Waves is a state-of-the-art 21st century warehouse in PKT Logistics Group Sdn Bhd’s integrated sustainable hub strategically located in the Batu Kawan Industrial Park on mainland Penang.The Malaysia Book of Records recognised it as both ‘The Longest Warehouse in Malaysia’ (648m) and ‘The Biggest Single Storey Warehouse in Malaysia’ (650,000sq ft) in August 2018.On the accolade, Group chief executive and managing director Datuk Michael Tio shared “We’re honoured by the glo

https://www.thestar.com.my/starpicks/2021/06/01/making-sense-of-the-complexities-of-media-and-communications
Tuesday, 01 Jun 2021
8:15 AM MYT
Making sense of the complexities of media and communications
StarPicks

IN increasingly convoluted times and a perpetually evolving world, communication has become – more than ever – an important part of society.Monash University Malaysia’s Master of Communications and Media Studies (MCMS) co-ordinator Dr Tan Meng Yoe (pic) provides insights into the programme and relevance of communications and media studies in an increasingly complex world:“Communications is more than what is presented to us. We are inundated with media content from various sources, both solicited and unsolicited.“Looking beyond the noise, we must find out what’s really going on. It is in this process of inquiry where important critical thinking skills are developed. In news, we have to ask: whose agenda is prioritised, how stories are framed, what version of reality is laid ou

https://www.thestar.com.my/starpicks/2021/05/31/fintech---the-new-normal-in-finance
Monday, 31 May 2021
8:26 AM MYT
FinTech – the ‘new normal’ in finance
StarPicks

THE Fourth Industrial Revolution (IR4.0) is changing how we live, work and communicate.With this development that focuses heavily on interconnectivity, automation and real-time data, intelligent technologies are rapidly transforming almost every facet of our life.Cash is no longer king, as banknotes are now being replaced by digital currencies, while e-wallets and digital payments are the new forms of payment.These transformations are spurred by the application of financial technology (FinTech) which embodies IR4.0 technologies such as blockchain, artificial intelligence (AI) and data analytics.FinTech has risen to challenges posed by the Covid-19 pandemic by delivering digital, secure, seamless user experiences.Research conducted by the University of Oxford found that accountants have a 95% chance of being impacted by AI t

https://www.thestar.com.my/starpicks/2021/05/31/the-secure-engineering-career-option-for-the-digital-age
Monday, 31 May 2021
8:06 AM MYT
The secure engineering career option for the digital age
StarPicks

COMPUTER engineers are the drivers behind many of the innovations in technology today, from apps on mobile devices and computers to the latest computer platforms.And the first step towards a career in computer engineering begins at Universiti Teknologi Petronas (UTP).Existing at the crossroads of computer science and electrical engineering, computer engineers have been behind many, if not all, technological advancements of the computer age.From computers, the Internet and smart phones to the technologies that are driving the Fourth Industrial Revolution (IR 4.0) such as Big Data, Internet of Things (IoT), cloud computing, autonomous systems and machine learning, computer engineers have made their presence felt.Although computer engineers have been around for a long time, in the past t

### Store the crawled data into data frame

In [11]:
current = pd.DataFrame(news_list)
current.head()

,Date,Time,Type,Author,Title,Content
0,"Wednesday, 09 Jun 2021",7:00 AM MYT,News,RAZAK AHMAD，DIYANA PFORDTEN,INTERACTIVE: Here’s what vaccinating Malaysian...,PETALING JAYA: Malaysia is expected to announc...
1,"Wednesday, 21 Oct 2020",8:00 AM MYT,News,DIYANA PFORDTEN，RAZAK AHMAD,Covid-19: Total cases worldwide,PETALING JAYA: The World Health Organization (...
2,"Saturday, 05 Jun 2021",7:00 AM MYT,News,RAZAK AHMAD,INTERACTIVE: How Malaysia’s fight against Covi...,"PETALING JAYA: A total of 3,330,436 Covid-19 v..."
3,"Wednesday, 02 Jun 2021",7:00 AM MYT,News,DIYANA PFORDTEN,INTERACTIVE: More Covid-19 patients are being ...,PETALING JAYA: The number of people in Malaysi...
4,"Tuesday, 25 May 2021",7:00 AM MYT,News,DIYANA PFORDTEN，RAZAK AHMAD,INTERACTIVE: Malaysian Twitter users rank near...,PETALING JAYA: Computer science graduate Muadz...


In [12]:
print("Shape    : ", current.shape)

Shape    :  (295, 6)


### Read older csv file

In [13]:
before = pd.read_csv('News.csv')
before.head()

,Date,Time,Type,Author,Title,Content
0,"Saturday, 05 Jun 2021",7:00 AM MYT,News,RAZAK AHMAD,INTERACTIVE: How Malaysia’s fight against Covi...,"PETALING JAYA: A total of 3,330,436 Covid-19 v..."
1,"Wednesday, 21 Oct 2020",8:00 AM MYT,News,DIYANA PFORDTEN，RAZAK AHMAD,Covid-19: Total cases worldwide,PETALING JAYA: The World Health Organization (...
2,"Wednesday, 02 Jun 2021",7:00 AM MYT,News,DIYANA PFORDTEN,INTERACTIVE: More Covid-19 patients are being ...,PETALING JAYA: The number of people in Malaysi...
3,"Tuesday, 25 May 2021",7:00 AM MYT,News,DIYANA PFORDTEN，RAZAK AHMAD,INTERACTIVE: Malaysian Twitter users rank near...,PETALING JAYA: Computer science graduate Muadz...
4,"Sunday, 23 May 2021",NaN,News,NaN,INTERACTIVE: SUFFERING IN SILENCE,Suffering In SilenceHow the Pandemic is Affect...


In [14]:
print("Shape    : ", before.shape)

Shape    :  (929, 6)


### Combine 2 data frames

In [17]:
# Concatenate current and older data
data = pd.concat([before, current])
# Drop duplicates between previous and current data
data.drop_duplicates(inplace = True, subset = ['Content'])
# Store the data back to the News csv file
data.to_csv('News.csv', index=False)
data.head()

,Date,Time,Type,Author,Title,Content
0,"Saturday, 05 Jun 2021",7:00 AM MYT,News,RAZAK AHMAD,INTERACTIVE: How Malaysia’s fight against Covi...,"PETALING JAYA: A total of 3,330,436 Covid-19 v..."
1,"Wednesday, 21 Oct 2020",8:00 AM MYT,News,DIYANA PFORDTEN，RAZAK AHMAD,Covid-19: Total cases worldwide,PETALING JAYA: The World Health Organization (...
2,"Wednesday, 02 Jun 2021",7:00 AM MYT,News,DIYANA PFORDTEN,INTERACTIVE: More Covid-19 patients are being ...,PETALING JAYA: The number of people in Malaysi...
3,"Tuesday, 25 May 2021",7:00 AM MYT,News,DIYANA PFORDTEN，RAZAK AHMAD,INTERACTIVE: Malaysian Twitter users rank near...,PETALING JAYA: Computer science graduate Muadz...
4,"Sunday, 23 May 2021",NaN,News,NaN,INTERACTIVE: SUFFERING IN SILENCE,Suffering In SilenceHow the Pandemic is Affect...


In [18]:
print("Rows     : ", data.shape[0])
print("Columns  : ", data.shape[1])
print("Shape    : ", data.shape)
print("Features : ", data.columns.tolist())

Rows     :  1022
Columns  :  6
Shape    :  (1022, 6)
Features :  ['Date', 'Time', 'Type', 'Author', 'Title', 'Content']
